In [1]:
# Header block to include all modules that must be imported ahead of time
# Only needs to be run once per session, and each time a new module is added

import requests
import json
import pandas as pd # this will need to be installed via command line first

In [2]:
# Method to get the salary cap hits for every player currently signed on an NFL team
# Creates a csv with columns player name, salary cap hit, and team location abbrviation 

def GetTeamSalary(team: list): # accepts a list ['team-name', 'LOC']
    # Use requests to get the raw HTML response from overthecap.com
    OTCResponse = requests.get(f'https://overthecap.com/salary-cap/{team[0]}')
        
    # If we get a normal response proceed with data scraping
    if OTCResponse.status_code == 200:
        print(f"Request succeeded with status code {OTCResponse.status_code}")
        TeamNameCaptalized = (team[0].replace("-", " ")).title()
        print(f"Here are the Salary Cap hits for all players who are currently signed with the {TeamNameCaptalized}:")

        # Pandas read_html method allows the table contents to be put into a DataFrame
        TableTag = 'salary-cap-table contracted-players'
        DataFrames = pd.read_html(OTCResponse.content, attrs = {'class': TableTag})
        
        # Select the specific parts of the dataframe we want 
        SalariesDF = DataFrames[0][['Player', 'Cap Number']]
        if len(SalariesDF) >= 51:
            SalariesDF = SalariesDF.drop(51, axis=0)
        # Add team location abv to each player
        SalariesDF = SalariesDF.assign(TeamLOC = team[1])
        SalariesDF = SalariesDF.assign(TeamName = (team[0].replace("-", " ")).title())
        
        
        # Change the order of columns to make more sense semantically 
        new_order = ['Player', 'TeamLOC', 'TeamName', 'Cap Number']
        SalariesDF = SalariesDF.loc[:, new_order]
        
        # Specify filename and path
        csvPathName = f'./data/team-salary-lists/{TeamNameCaptalized}-Player-Salary-List.csv'
        
        # Create a .csv file with all the player names and their cap hits
        SalariesDF.to_csv(csvPathName, index = False, header = False)
        
        # Read the csv back for testing purposes
        SalariesCSV = pd.read_csv(f'./data/team-salary-lists/{TeamNameCaptalized}-Player-Salary-List.csv', header = None)
        SalariesCSV.columns = ['Player', 'TeamLOC', 'TeamName', 'Cap Number']
        print(SalariesCSV)
        
    # If we don't get a normal reponse, stop scraping
    else:
        print(f"Request failed with status code {OTCResponse.status_code}")
        return -1


In [3]:
# Method to get the salary cap hits for every player currently signed on every NFL team
# Reads in the team list csv, and calls the GetTeamSalaryCapHits method for each team
def GetAllTeamSalary():
    # Read in the full team list csv
    FullTeamListCSV = pd.read_csv('./data/team-attribute-lists/Teams-List.csv', header = None)
    FullTeamListCSV.columns = ['TeamName', 'Abbreviation', 'Conference', 'Division']
    print(FullTeamListCSV)
    
    # Create a list of all the team names and abbreviations as a list of lists
    TeamList = []
    FullTeamListCSV.apply(lambda row: TeamList.append(row[:2].tolist()), axis = 1)
    
    # Get the salary cap hits for each team
    for team in TeamList:
        # format the team name to be used in the URL
        team[0] = team[0].replace(" ", "-").lower()
        GetTeamSalary(team)
    

GetAllTeamSalary()

                 TeamName Abbreviation Conference Division
0       Arizona Cardinals          ARI        NFC     West
1         Atlanta Falcons          ATL        NFC    South
2        Baltimore Ravens          BAL        AFC    North
3           Buffalo Bills          BUF        AFC     East
4       Carolina Panthers          CAR        NFC    South
5           Chicago Bears          CHI        NFC    North
6      Cincinnati Bengals          CIN        AFC    North
7        Cleveland Browns          CLE        AFC    North
8          Dallas Cowboys          DAL        NFC     East
9          Denver Broncos          DEN        AFC     West
10          Detroit Lions          DET        NFC    North
11      Green Bay Packers           GB        NFC    North
12         Houston Texans          HOU        AFC    South
13     Indianapolis Colts          IND        AFC    South
14   Jacksonville Jaguars          JAC        AFC    South
15     Kansas City Chiefs           KC        AFC     We

Request succeeded with status code 200
Here are the Salary Cap hits for all players who are currently signed with the Cleveland Browns:
                  Player TeamLOC          TeamName   Cap Number
0         Deshaun Watson     CLE  Cleveland Browns  $63,774,678
1           Amari Cooper     CLE  Cleveland Browns  $23,776,000
2          Myles Garrett     CLE  Cleveland Browns  $20,168,895
3            Denzel Ward     CLE  Cleveland Browns  $12,138,918
4           Joel Bitonio     CLE  Cleveland Browns  $12,135,000
..                   ...     ...               ...          ...
85          Vincent Gray     CLE  Cleveland Browns     $795,000
86         Caleb Johnson     CLE  Cleveland Browns     $795,000
87          Roy Mbaeteka     CLE  Cleveland Browns     $795,000
88          Matt Landers     CLE  Cleveland Browns     $795,000
89  Zaire Mitchell-Paden     CLE  Cleveland Browns     $795,000

[90 rows x 4 columns]
Request succeeded with status code 200
Here are the Salary Cap hits for a

Request succeeded with status code 200
Here are the Salary Cap hits for all players who are currently signed with the Los Angeles Chargers:
            Player TeamLOC              TeamName   Cap Number
0        Joey Bosa     LAC  Los Angeles Chargers  $26,111,666
1      Khalil Mack     LAC  Los Angeles Chargers  $25,395,000
2     Derwin James     LAC  Los Angeles Chargers  $19,856,494
3   Justin Herbert     LAC  Los Angeles Chargers  $19,345,675
4     Trey Pipkins     LAC  Los Angeles Chargers   $8,750,000
..             ...     ...                   ...          ...
86    Matt Hankins     LAC  Los Angeles Chargers     $795,000
87     Brent Laing     LAC  Los Angeles Chargers     $795,000
88      C.J. Okoye     LAC  Los Angeles Chargers     $795,000
89    Chris Wilcox     LAC  Los Angeles Chargers     $795,000
90       Ty Shelby     LAC  Los Angeles Chargers     $795,000

[91 rows x 4 columns]
Request succeeded with status code 200
Here are the Salary Cap hits for all players who are c

Request succeeded with status code 200
Here are the Salary Cap hits for all players who are currently signed with the San Francisco 49Ers:
             Player TeamLOC             TeamName   Cap Number
0    Trent Williams      SF  San Francisco 49Ers  $31,568,542
1      Deebo Samuel      SF  San Francisco 49Ers  $28,633,765
2   Charvarius Ward      SF  San Francisco 49Ers  $18,401,000
3         Nick Bosa      SF  San Francisco 49Ers  $14,671,000
4     Brandon Aiyuk      SF  San Francisco 49Ers  $14,124,000
..              ...     ...                  ...          ...
86   Jaylen Mahoney      SF  San Francisco 49Ers     $797,666
87   Tayler Hawkins      SF  San Francisco 49Ers     $795,000
88    Isaac Alarcon      SF  San Francisco 49Ers     $795,000
89     Shakel Brown      SF  San Francisco 49Ers     $795,000
90       Tay Martin      SF  San Francisco 49Ers     $795,000

[91 rows x 4 columns]
Request succeeded with status code 200
Here are the Salary Cap hits for all players who are cu